In [1]:
import sys
import contextlib
import transformers
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import os
import sys
import torch
import math
import pandas as pd
import numpy as np

from transformers import pipeline, set_seed
set_seed(42)



In [91]:
# dems_tokenizer = GPT2Tokenizer.from_pretrained('../models/gpt2_dems_full_data_single_follow/checkpoint-1230444/')
# dem_model = GPT2LMHeadModel.from_pretrained('../models/gpt2_dems_full_data_single_follow/checkpoint-1230444/')

# repub_tokenizer = GPT2Tokenizer.from_pretrained('../models/gpt2_repubs_full_data_single_follow/checkpoint-490180/')
# repub_model = GPT2LMHeadModel.from_pretrained('../models/gpt2_repubs_full_data_single_follow/checkpoint-490180/')

In [24]:

# https://huggingface.co/blog/how-to-generate

def generate_with_a_prompt(prompt, text_gen_pipeline):
    """
    Generate a list of statements given the prompt based on one GPT-2 model
    
    TODO:
        - temperature
    
    NOTES:
        1. 50256 corresponds to '<|endoftext|>'
    """
    
    results = text_gen_pipeline(prompt, 
                                max_length=30,
                                temperature=0.5,
                                num_return_sequences=100,
                                pad_token_id=50256,
                                clean_up_tokenization_spaces=True
                               )
    results = [res['generated_text'].split("\n")[0] for res in results]
    return results

In [3]:
# dem_generator = pipeline('text-generation', model='../models/gpt2_dems_full_data_single_follow/', device=0)
dem_generator = pipeline('text-generation', model='../models/pretrained_gpt2_dems_4M_full_data_single_follow/', device=0)

repub_generator = pipeline('text-generation', model='../models/gpt2_reps_full_data/', device=0)

In [5]:
sentiment_model = pipeline("sentiment-analysis",
                           model="cardiffnlp/twitter-roberta-base-sentiment-latest", 
                           tokenizer="cardiffnlp/twitter-roberta-base-sentiment-latest")

Downloading:   0%|          | 0.00/929 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/478M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [34]:
def aggregate_opinion(prompt, text_gen, SentimentModel):
#     Sentiment2score = {
#         "Positive": 1,
#         "Neutral": 0,
#         "Negative": -1
#     }
    
    Sentiment2score = {
    "Negative": 0,
    "Positive": 100,
    "Neutral": 50
    }
    
    results = generate_with_a_prompt(prompt, text_gen)
    preds = SentimentModel(results)
    scores = [Sentiment2score[pred['label']] for pred in preds]
    
    return np.array(scores).mean(), preds

In [35]:
df = pd.read_csv("./anes2020_pilot_prompt_probing.csv")
df.head(4)

,Unnamed: 0,Democrat,Republican,is_repub_leading,Prompt1,Prompt2,Prompt3,Prompt4
0,fttrump1,17.664378,77.833163,True,Donald Trump,Donald Trump is,Donald Trump is a,Donald Trump is the
1,ftobama1,81.292704,29.992821,False,Barack Obama,Barack Obama is,Barack Obama is a,Barack Obama is the
2,ftbiden1,66.497423,24.401235,False,Joe Biden,Joe Biden is,Joe Biden is a,Joe Biden is the
3,ftwarren1,59.843478,20.457680,False,Elizabeth Warren,Elizabeth Warren is,Elizabeth Warren is a,Elizabeth Warren is the


In [36]:
# df = pd.read_csv("../data/prompts/partisan_prefix_prompts.txt", header=None)

pred_labels = []
pred_avg_sentiments = []
gold_labels = []
prompts = []
prompts
for prompt, gold_label in df[["Prompt4", 'is_repub_leading']].values:
    prompt = prompt.strip()
    gold_label = int(gold_label)
    # gold_label = {"positive": 1, "negative": 0}[gold_label]
    
    avg_dem_sentiment, _ = aggregate_opinion(prompt, dem_generator, sentiment_model)
    avg_repub_sentiment, _ = aggregate_opinion(prompt, repub_generator, sentiment_model)
    
    pred_avg_sentiments.append((avg_repub_sentiment, avg_dem_sentiment))
    if avg_repub_sentiment > avg_dem_sentiment:
        pred_label = 1
    else:
        pred_label = 0
        
    pred_labels.append(pred_label)
    gold_labels.append(gold_label)
    prompts.append(prompt)
    print(prompt)
    print("Gold Label:", gold_label)
    print("Pred Label:", pred_label)
    print()

/home/hjian42/.conda/envs/change/lib/python3.8/site-packages/transformers/pipelines/base.py:997: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Donald Trump is the
Gold Label: 1
Pred Label: 1

Barack Obama is the
Gold Label: 0
Pred Label: 0

Joe Biden is the
Gold Label: 0
Pred Label: 0

Elizabeth Warren is the
Gold Label: 0
Pred Label: 0

Bernie Sanders is the
Gold Label: 0
Pred Label: 0

Pete Buttigieg is the
Gold Label: 0
Pred Label: 0

Kamala Harris is the
Gold Label: 0
Pred Label: 0

Amy Klobuchar is the
Gold Label: 0
Pred Label: 0

Mike Pence is the
Gold Label: 1
Pred Label: 1

Andrew Yang is the
Gold Label: 0
Pred Label: 0

Nancy Pelosi is the
Gold Label: 0
Pred Label: 0

Marco Rubio is the
Gold Label: 1
Pred Label: 1

Alexandria Ocasio-Cortez is the
Gold Label: 0
Pred Label: 0

Nikki Haley is the
Gold Label: 1
Pred Label: 1

Clarence Thomas is the
Gold Label: 1
Pred Label: 1

Dr. Anthony Fauci is the
Gold Label: 0
Pred Label: 0

black people are the
Gold Label: 0
Pred Label: 0

white people are the
Gold Label: 1
Pred Label: 0

Hispanic people are the
Gold Label: 0
Pred Label: 0

Asian people are the
Gold Label: 0
Pred L

In [26]:
from sklearn.metrics import accuracy_score

accuracy_score(gold_labels, pred_labels)

0.9

In [37]:
pred_avg_sentiments

[(80.5, 6.5),
 (5.0, 69.5),
 (10.5, 78.5),
 (56.5, 85.0),
 (62.0, 79.0),
 (43.5, 81.0),
 (18.0, 88.5),
 (26.5, 53.5),
 (50.5, 11.5),
 (48.0, 73.0),
 (1.0, 44.0),
 (53.0, 18.5),
 (34.0, 57.5),
 (51.5, 34.0),
 (71.0, 56.0),
 (45.5, 73.0),
 (4.0, 10.0),
 (2.5, 5.5),
 (9.5, 16.0),
 (9.0, 7.0),
 (0.5, 2.5),
 (2.5, 9.5),
 (29.5, 64.5),
 (15.5, 15.0),
 (2.5, 5.5),
 (5.5, 2.0),
 (33.0, 31.5),
 (37.5, 52.0),
 (5.0, 0.5),
 (0.0, 19.5)]

In [27]:
avg_sentiment, results = aggregate_opinion("Fauci is", dem_generator, sentiment_model)
avg_sentiment

/home/hjian42/.conda/envs/change/lib/python3.8/site-packages/transformers/pipelines/base.py:997: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


0.32

In [74]:
avg_sentiment, _ = aggregate_opinion("Fauci is", repub_generator, sentiment_model)
avg_sentiment

-0.9

In [33]:
from collections import Counter


66.0

[{'label': 'Negative', 'score': 0.8878867030143738},
 {'label': 'Negative', 'score': 0.8414068222045898},
 {'label': 'Negative', 'score': 0.9111444354057312},
 {'label': 'Negative', 'score': 0.9114534854888916},
 {'label': 'Negative', 'score': 0.9371337890625},
 {'label': 'Negative', 'score': 0.8892924189567566},
 {'label': 'Negative', 'score': 0.8128095865249634},
 {'label': 'Negative', 'score': 0.8801783323287964},
 {'label': 'Negative', 'score': 0.9490361213684082},
 {'label': 'Negative', 'score': 0.9414410591125488}]